In [1]:
!cp -r /kaggle/input/evaluate-stargan-graph/* /kaggle/working/


In [2]:
import os, random, shutil
from pathlib import Path

# --- các đường dẫn (chỉnh nếu khác) ---
WORK_STARGAN = "/kaggle/working/stargan"
SOURCE_IMG_DIR = "/kaggle/input/celeba-dataset/img_align_celeba/img_align_celeba"
DEST_IMG_DIR = os.path.join(WORK_STARGAN, "data/celeba/img")
ATTR_SRC = "/kaggle/input/celeba-dataset/list_attr_celeba.txt"  # original attr file in input
ATTR_DEST = os.path.join(WORK_STARGAN, "data/celeba/list_attr_celeba.txt")
ATTR_SMALL = os.path.join(WORK_STARGAN, "data/celeba/1k_attr_face.txt")
NUM_SELECT = 1000


In [3]:
import os, cv2, shutil, numpy as np, torch
from skimage.metrics import structural_similarity as ssim
from torchvision import transforms
from torchvision.models import inception_v3
from torchvision.models.feature_extraction import create_feature_extractor
from concurrent.futures import ProcessPoolExecutor
from PIL import Image
from scipy.linalg import sqrtm

# ============================================================
# CONFIG
# ============================================================
PARTS = 18              # 1 real + 17 fake
PART_W, PART_H = 156, 156
NUM_CRITERIA = 17
INCEPTION_ON_CPU = True

device = 'cuda' if (torch.cuda.is_available() and not INCEPTION_ON_CPU) else 'cpu'

# ============================================================
# LOAD INCEPTION (fixed)
# ============================================================
from torchvision.models import Inception_V3_Weights
weights = Inception_V3_Weights.IMAGENET1K_V1

# KHÔNG truyền aux_logits nữa
inc = inception_v3(weights=weights).to(device).eval()

feat_extractor = create_feature_extractor(inc, return_nodes={'avgpool': 'feat'})

transform_incep = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# ============================================================
# IMAGE SPLIT
# ============================================================
def split_image_horizontally(image):
    parts = []
    for i in range(PARTS):
        x = i * PART_W
        crop = image[0:PART_H, x:x + PART_W]
        parts.append(crop.copy())
    return parts

def save_real_and_fakes_from_montage(montage_path, out_dir):
    img = cv2.imread(montage_path)
    if img is None:
        return None, []

    parts = split_image_horizontally(img)
    fname = os.path.basename(montage_path)

    real_dir = os.path.join(out_dir, "real")
    os.makedirs(real_dir, exist_ok=True)

    real_path = os.path.join(real_dir, f"real_{fname}")
    cv2.imwrite(real_path, parts[0])

    fake_paths = []
    for i, p in enumerate(parts[1:], start=1):
        fakedir = os.path.join(out_dir, "fake", str(i))
        os.makedirs(fakedir, exist_ok=True)
        fp = os.path.join(fakedir, f"fake_{fname}")
        cv2.imwrite(fp, p)
        fake_paths.append(fp)

    return real_path, fake_paths

# ============================================================
# SSIM
# ============================================================
def compute_ssim_pair(real_path, fake_path):
    a = cv2.imread(real_path)
    b = cv2.imread(fake_path)
    if a is None or b is None:
        return float('nan')

    if a.shape != b.shape:
        b = cv2.resize(b, (a.shape[1], a.shape[0]))

    try:
        return ssim(a, b, channel_axis=-1, win_size=3)
    except:
        return float('nan')

# ============================================================
# FID
# ============================================================
def extract_inception_feature(path):
    img = Image.open(path).convert('RGB')
    transform = transforms.Compose([
        transforms.Resize(299),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )
    ])
    x = transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        feat = inc(x)  # dùng model inception_v3 gốc
        feat = feat.squeeze().cpu().numpy()

    return feat

def calculate_fid(real_features, fake_features):
    mu_real = real_features.mean(axis=0)
    mu_fake = fake_features.mean(axis=0)

    sigma_real = np.cov(real_features, rowvar=False)
    sigma_fake = np.cov(fake_features, rowvar=False)

    diff = mu_real - mu_fake

    # matrix square root
    covmean = sqrtm(sigma_real @ sigma_fake)
    if np.iscomplexobj(covmean):
        covmean = covmean.real

    fid = diff @ diff + np.trace(sigma_real + sigma_fake - 2 * covmean)
    return float(fid)


# ============================================================
# FULL EVALUATION
# ============================================================
def evaluate_results_folder(result_dir, max_workers=8):

    # 1. xóa real/fake cũ
    for sub in ["real", "fake"]:
        p = os.path.join(result_dir, sub)
        if os.path.exists(p):
            shutil.rmtree(p)
    os.makedirs(result_dir, exist_ok=True)

    # 2. tách montage
    montage_files = sorted(
        [os.path.join(result_dir, f) for f in os.listdir(result_dir) if f.lower().endswith(".jpg")]
    )

    for m in montage_files:
        save_real_and_fakes_from_montage(m, result_dir)

    # 3. load danh sách real
    real_dir = os.path.join(result_dir, "real")
    real_files = sorted(os.listdir(real_dir))
    real_paths = [os.path.join(real_dir, f) for f in real_files]

    # 4. SSIM
    ssim_avgs = []
    with ProcessPoolExecutor(max_workers=max_workers) as ex:
        for i in range(1, NUM_CRITERIA + 1):
            fake_dir = os.path.join(result_dir, "fake", str(i))
            if not os.path.exists(fake_dir):
                ssim_avgs.append(float('nan'))
                continue

            fake_files = sorted(os.listdir(fake_dir))
            fake_paths = [os.path.join(fake_dir, f) for f in fake_files]

            n = min(len(real_paths), len(fake_paths))
            jobs = [ex.submit(compute_ssim_pair, real_paths[k], fake_paths[k]) for k in range(n)]
            vals = [j.result() for j in jobs]
            ssim_avgs.append(float(np.nanmean(vals)))

    # 5. FID
    fid_scores = []
    for i in range(1, NUM_CRITERIA + 1):
        fake_dir = os.path.join(result_dir, "fake", str(i))

        if not os.path.exists(fake_dir):
            fid_scores.append(float('nan'))
            continue

        fake_files = sorted(os.listdir(fake_dir))
        fake_paths = [os.path.join(fake_dir, f) for f in fake_files]

        n = min(len(real_paths), len(fake_paths))
        real_feats = np.array([extract_inception_feature(real_paths[k]) for k in range(n)])
        fake_feats = np.array([extract_inception_feature(fake_paths[k]) for k in range(n)])
        
        fid_scores.append(calculate_fid(real_feats, fake_feats))

    return (
        float(np.nanmean(ssim_avgs)),
        ssim_avgs,
        float(np.nanmean([x for x in fid_scores if not np.isnan(x)])),
        fid_scores
    )


Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth
100%|██████████| 104M/104M [00:00<00:00, 153MB/s] 
/usr/local/lib/python3.11/dist-packages/torchvision/models/feature_extraction.py:175: UserWarning: NOTE: The nodes obtained by tracing the model in eval mode are a subsequence of those obtained in train mode. When choosing nodes for feature extraction, you may need to specify output nodes for train and eval mode separately.
  warnings.warn(msg + suggestion_msg)


In [4]:
import subprocess, time, pandas as pd, shutil

CHECKPOINTS = [100000, 150000, 200000]
RESULT_DIR = "/kaggle/working/stargan/stargan_celeba/results"
MODEL_SAVE_DIR = "/kaggle/working/stargan/stargan_celeba/models"
ATTR_PATH = os.path.join(WORK_STARGAN, "data/celeba/1k_attr_face.txt")
CELEBA_IMG_DIR = os.path.join(WORK_STARGAN, "data/celeba/img")

master_results = []

criteria_names = [
    "Hói Đầu", "Tóc Mái", "Tóc Đen", "Tóc Vàng", "Mũm Mĩm", "Đeo Kính",
    "Râu Dê", "Tóc Bạc", "Trang Điểm Đậm", "Nam Giới", "Miệng Hơi Mở",
    "Râu Mép", "Không Râu", "Da Nhợt Nhạt", "Má Hồng", "Mỉm Cười", "Đánh Son"
]

for ckpt in CHECKPOINTS:
    print("\n" + "="*60)
    print(f"Testing checkpoint {ckpt}")

    if os.path.exists(RESULT_DIR):
        shutil.rmtree(RESULT_DIR)
    os.makedirs(RESULT_DIR, exist_ok=True)

    cmd = [
        "python", os.path.join(WORK_STARGAN, "main.py"),
        "--mode", "test",
        "--dataset", "CelebA",
        "--image_size", "156",
        "--c_dim", "17",
        "--selected_attrs",
        "Bald", "Bangs", "Black_Hair", "Blond_Hair", "Chubby", "Eyeglasses",
        "Goatee", "Gray_Hair", "Heavy_Makeup", "Male", "Mouth_Slightly_Open",
        "Mustache", "No_Beard", "Pale_Skin", "Rosy_Cheeks", "Smiling", "Wearing_Lipstick",
        f"--model_save_dir={MODEL_SAVE_DIR}",
        f"--result_dir={RESULT_DIR}",
        f"--attr_path={ATTR_PATH}",
        f"--celeba_image_dir={CELEBA_IMG_DIR}",
        f"--test_iters={ckpt}",
        "--batch_size=1"
    ]

    print("Running:", " ".join(cmd))
    t0 = time.time()
    subprocess.run(cmd, check=True)
    print(f"Test done in {time.time()-t0:.1f}s")

    # evaluate
    print("Evaluating SSIM/FID ...")
    t0 = time.time()
    mean_ssim, ssim_attr, mean_fid, fid_attr = evaluate_results_folder(RESULT_DIR, max_workers=8)
    print(f"Eval done in {time.time()-t0:.1f}s | SSIM={mean_ssim:.4f} | FID={mean_fid:.2f}")

    record = {
        "checkpoint": ckpt,
        "mean_ssim": mean_ssim,
        "mean_fid": mean_fid
    }

    for i, name in enumerate(criteria_names):
        record[f"ssim_{i+1}"] = float(ssim_attr[i])
        record[f"fid_{i+1}"] = float(fid_attr[i])

    master_results.append(record)

    df = pd.DataFrame(master_results)
    df.to_csv("/kaggle/working/stargan_eval_results_master.csv", index=False)
    print("Saved to /kaggle/working/stargan_eval_results_master.csv")



Testing checkpoint 100000
Running: python /kaggle/working/stargan/main.py --mode test --dataset CelebA --image_size 156 --c_dim 17 --selected_attrs Bald Bangs Black_Hair Blond_Hair Chubby Eyeglasses Goatee Gray_Hair Heavy_Makeup Male Mouth_Slightly_Open Mustache No_Beard Pale_Skin Rosy_Cheeks Smiling Wearing_Lipstick --model_save_dir=/kaggle/working/stargan/stargan_celeba/models --result_dir=/kaggle/working/stargan/stargan_celeba/results --attr_path=/kaggle/working/stargan/data/celeba/1k_attr_face.txt --celeba_image_dir=/kaggle/working/stargan/data/celeba/img --test_iters=100000 --batch_size=1


2025-12-01 11:04:18.407370: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764587058.756166      44 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764587058.872053      44 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'
AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'
AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'
AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'
AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'
2025-12-01 11:04:41.464305: E external/local_xla/xla/stream_executor/cuda/cud

Namespace(c_dim=17, c2_dim=8, celeba_crop_size=178, rafd_crop_size=256, image_size=156, g_conv_dim=64, d_conv_dim=64, g_repeat_num=6, d_repeat_num=6, lambda_cls=1, lambda_rec=10, lambda_gp=10, dataset='CelebA', batch_size=1, num_iters=200000, num_iters_decay=100000, g_lr=0.0001, d_lr=0.0001, n_critic=5, beta1=0.5, beta2=0.999, resume_iters=None, selected_attrs=['Bald', 'Bangs', 'Black_Hair', 'Blond_Hair', 'Chubby', 'Eyeglasses', 'Goatee', 'Gray_Hair', 'Heavy_Makeup', 'Male', 'Mouth_Slightly_Open', 'Mustache', 'No_Beard', 'Pale_Skin', 'Rosy_Cheeks', 'Smiling', 'Wearing_Lipstick'], test_iters=100000, num_workers=1, mode='test', use_tensorboard=True, celeba_image_dir='/kaggle/working/stargan/data/celeba/img', attr_path='/kaggle/working/stargan/data/celeba/1k_attr_face.txt', rafd_image_dir='data/RaFD/train', log_dir='stargan/logs', model_save_dir='/kaggle/working/stargan/stargan_celeba/models', sample_dir='stargan/samples', result_dir='/kaggle/working/stargan/stargan_celeba/results', log_s

2025-12-01 14:49:49.828454: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764600589.893998      83 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764600589.915856      83 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'
AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'
AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'
AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'
AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'
2025-12-01 14:49:59.939528: E external/local_xla/xla/stream_executor/cuda/cud

Namespace(c_dim=17, c2_dim=8, celeba_crop_size=178, rafd_crop_size=256, image_size=156, g_conv_dim=64, d_conv_dim=64, g_repeat_num=6, d_repeat_num=6, lambda_cls=1, lambda_rec=10, lambda_gp=10, dataset='CelebA', batch_size=1, num_iters=200000, num_iters_decay=100000, g_lr=0.0001, d_lr=0.0001, n_critic=5, beta1=0.5, beta2=0.999, resume_iters=None, selected_attrs=['Bald', 'Bangs', 'Black_Hair', 'Blond_Hair', 'Chubby', 'Eyeglasses', 'Goatee', 'Gray_Hair', 'Heavy_Makeup', 'Male', 'Mouth_Slightly_Open', 'Mustache', 'No_Beard', 'Pale_Skin', 'Rosy_Cheeks', 'Smiling', 'Wearing_Lipstick'], test_iters=150000, num_workers=1, mode='test', use_tensorboard=True, celeba_image_dir='/kaggle/working/stargan/data/celeba/img', attr_path='/kaggle/working/stargan/data/celeba/1k_attr_face.txt', rafd_image_dir='data/RaFD/train', log_dir='stargan/logs', model_save_dir='/kaggle/working/stargan/stargan_celeba/models', sample_dir='stargan/samples', result_dir='/kaggle/working/stargan/stargan_celeba/results', log_s

2025-12-01 18:25:16.176628: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764613516.230253     122 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764613516.247540     122 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'
AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'
AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'
AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'
AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'
2025-12-01 18:25:26.099000: E external/local_xla/xla/stream_executor/cuda/cud

Namespace(c_dim=17, c2_dim=8, celeba_crop_size=178, rafd_crop_size=256, image_size=156, g_conv_dim=64, d_conv_dim=64, g_repeat_num=6, d_repeat_num=6, lambda_cls=1, lambda_rec=10, lambda_gp=10, dataset='CelebA', batch_size=1, num_iters=200000, num_iters_decay=100000, g_lr=0.0001, d_lr=0.0001, n_critic=5, beta1=0.5, beta2=0.999, resume_iters=None, selected_attrs=['Bald', 'Bangs', 'Black_Hair', 'Blond_Hair', 'Chubby', 'Eyeglasses', 'Goatee', 'Gray_Hair', 'Heavy_Makeup', 'Male', 'Mouth_Slightly_Open', 'Mustache', 'No_Beard', 'Pale_Skin', 'Rosy_Cheeks', 'Smiling', 'Wearing_Lipstick'], test_iters=200000, num_workers=1, mode='test', use_tensorboard=True, celeba_image_dir='/kaggle/working/stargan/data/celeba/img', attr_path='/kaggle/working/stargan/data/celeba/1k_attr_face.txt', rafd_image_dir='data/RaFD/train', log_dir='stargan/logs', model_save_dir='/kaggle/working/stargan/stargan_celeba/models', sample_dir='stargan/samples', result_dir='/kaggle/working/stargan/stargan_celeba/results', log_s